In [13]:
import json
import prepro

periodLengthMS = 1
sampleRateUS = 1


with open('datav1.json', 'r') as f:
    dsds = json.load(f)


In [14]:
labels = ["Gerade", "Kopfhaken", "Kinnhaken"]
label_counts = {label: 0 for label in labels}

for item in dsds:
    if item['label'] in labels:
        label_counts[item['label']] += 1

for label, count in label_counts.items():
    print(f"Label: {label}, Count: {count}")


Label: Gerade, Count: 792
Label: Kopfhaken, Count: 736
Label: Kinnhaken, Count: 829


In [36]:
total_time_ns = sum(item['periodNS'] for item in dsds)
total_time_s = total_time_ns / 1e9 /3600
print(total_time_s)

1.072874183090268
